In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import torch
import torchvision
from torchvision import datasets
from torchvision import transforms
from torch.autograd import Variable
import torch.nn as nn
from tqdm import tqdm
import time

import os
from PIL import Image
from torch.utils.data import Dataset
import cv2
import torch.nn.functional as F
import torchvision.models as models


In [ ]:
# def gen_edge_mask(mask):
#     # Convert mask to binary image
#     mask = cv2.threshold(mask, 0.5, 1, cv2.THRESH_BINARY)[1]
    
#     # Compute edges using the Canny edge detection algorithm
#     edges = cv2.Canny(np.uint8(mask * 255), 100, 200)
    
#     # Dilate edges to make them thicker
#     kernel = np.ones((3, 3), np.uint8)
#     edges = cv2.dilate(edges, kernel, iterations=1)
    
#     # Return the edge mask
#     return edges

In [2]:
transform = transforms.Compose([
                    transforms.ToTensor(), 
            ])

trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset=datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=10,shuffle=True, num_workers=3)

testloader = torch.utils.data.DataLoader(testset,batch_size=10,shuffle=True,num_workers=3)

##### Bayar CNN or Constrained CNN

In [4]:
class BayarConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=5, stride=1, padding=0):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.minus1 = (torch.ones(self.in_channels, self.out_channels, 1) * -1.000)

        super(BayarConv2d, self).__init__()
        # only (kernel_size ** 2 - 1) trainable params as the center element is always -1
        self.kernel = nn.Parameter(torch.rand(self.in_channels, self.out_channels, kernel_size ** 2 - 1),
                                   requires_grad=True)


    def bayarConstraint(self):
        self.kernel.data = self.kernel.permute(2, 0, 1)
        self.kernel.data = torch.div(self.kernel.data, self.kernel.data.sum(0))
        self.kernel.data = self.kernel.permute(1, 2, 0)
        ctr = self.kernel_size ** 2 // 2
        real_kernel = torch.cat((self.kernel[:, :, :ctr], self.minus1.to(self.kernel.device), self.kernel[:, :, ctr:]), dim=2)
        real_kernel = real_kernel.reshape((self.out_channels, self.in_channels, self.kernel_size, self.kernel_size))
        return real_kernel

    def forward(self, x):
        x = F.conv2d(x, self.bayarConstraint(), stride=self.stride, padding=self.padding)
        return x

In [5]:
class BayarConv(nn.Module):
    def __init__(self):
        super(BayarConv, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=5, padding=2, bias=False)

        weight = self.conv1.weight.data
        sum_all_except_center = torch.sum(weight[:, :, :, :] - weight[:, :, 2:3, 2:3], dim=(1, 2, 3), keepdim=True)
        # weight[:, :, :, :] -= weight[:, :, 2:3, 2:3]
        weight[:, :, :, :] /= sum_all_except_center
        weight[:, :, 2:3, 2:3] = -1.0

        self.conv.weight.data[:, :, :, :] = weight

        

    def forward(self, x):
        
        x = self.conv1(x)


        with torch.no_grad():
            weight = self.conv1.weight.data
            sum_all_except_center = torch.sum(weight[:, :, :, :] - weight[:, :, 2:3, 2:3], dim=(1, 2, 3), keepdim=True)
            weight[:, :, :, :] -= weight[:, :, 2:3, 2:3]
            weight[:, :, :, :] /= sum_all_except_center
            weight[:, :, 2:3, 2:3] = -1.0
            self.conv.weight.data[:, :, :, :] = weight
            
        return x

##### ERB

In [6]:
class ERB(nn.Module):
    def __init__(self,in_channels,strides):
        super(ERB,self).__init__()

        self.relu=nn.ReLU()
        self.batch_norm=nn.BatchNorm2d(1)
        self.cnn_1=nn.Conv2d(in_channels,1,kernel_size=1,stride=strides,padding=0)
        self.cnn_2=nn.Conv2d(1,1,kernel_size=3,stride=strides,padding=1)

    def forward(self,x):
        temp=x
        temp=self.cnn_1(temp)
        x1=self.cnn_2(temp)
        x1=self.batch_norm(x1)
        x1=self.relu(x1)
        x1=self.cnn_2(x1)

        fin=temp+x1

        return fin

In [48]:
erb=ERB(512,1)
erb(torch.randn(10, 512, 4, 4) ).shape

torch.Size([10, 1, 4, 4])

##### Sobel

In [9]:
class Sobel_layer(nn.Module):
    def __init__(self,in_channels,out_channels):
        super(Sobel_layer, self).__init__()
        self.conv_x = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1,bias=False)
        self.batch_norm=nn.BatchNorm2d(out_channels)
        self.sigmoid=nn.Sigmoid()
        # self.normm=torch.norm()

        for j in range(out_channels):
            for i in range(in_channels):
                self.conv_x.weight.data[j, i, :, :] = torch.Tensor([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
        
        # self.conv_x.weight.data[1, 0, :, :] = torch.Tensor([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
        # self.conv_x.weight.data[1, 1, :, :] = torch.Tensor([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
        # self.conv_x.weight.data[1, 2, :, :] = torch.Tensor([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])

        self.conv_y = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1,bias=False)

        for j in range(out_channels):
            for i in range(in_channels):
                self.conv_y.weight.data[j, i, :, :] = torch.Tensor([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])
      
        # self.conv_y.weight.data[1, 0, :, :] = torch.Tensor([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])
        # self.conv_y.weight.data[1, 1, :, :] = torch.Tensor([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])
        # self.conv_y.weight.data[1, 2, :, :] = torch.Tensor([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])

    def forward(self, x):
        temp_x=self.conv_x(x)
        temp_x=self.batch_norm(temp_x)
        temp_y=self.conv_y(x)
        temp_y=self.batch_norm(temp_y)

        temp = torch.sqrt(torch.pow(temp_x, 2) + torch.pow(temp_y, 2))


        temp=self.sigmoid(temp)

        # print(temp)
        # print(x.shape)
        
        fin=temp*x
        return fin


##### Positional Attention Module

In [10]:
class Position(nn.Module):
    def __init__(self):
        super(Position,self).__init__()
        self.conv1=nn.Conv2d(4096,512,kernel_size=1,stride=1,padding=0)
        self.conv2=nn.Conv2d(4096,4096,kernel_size=1,stride=1,padding=0)

        self.sm=nn.Softmax()
        self.alpha = nn.Parameter(torch.tensor(0.0), requires_grad=True)
    
    def forward(self,x):
        w,h,c=x.size()
        temp_1=self.conv1(x)
        temp_1 = temp_1.reshape(-1, c)
        temp_2=torch.transpose(temp_1,0,1)
        position_attention_map=temp_1*temp_2

        position_attention_map=self.sm(position_attention_map)

        temp_2=self.conv2(x)
        temp_2 = temp_2.reshape(-1, c)
        position_attention_map=torch.transpose(position_attention_map,0,1)
        fin=temp_2*position_attention_map
        fin=fin.view(w,h,c)   
        return self.alpha*fin+x


##### Channel Attention Module

In [11]:
class Channel(nn.Module):
    def __init__(self):
        super(Channel,self).__init__()

        self.sm=nn.Softmax()
        self.beta = nn.Parameter(torch.tensor(0.0), requires_grad=True)
    
    def forward(self,x):
        w,h,c=x.size()
        temp_1 = x.reshape(-1, c)
        temp_2=torch.transpose(temp_1,0,1)
        channel_attention_map=temp_1*temp_2

        channel_attention_map=self.sm(channel_attention_map)

        channel_attention_map=torch.transpose(channel_attention_map,0,1)

        fin=temp_1*channel_attention_map
        fin=fin.view(w,h,c)   
        return self.beta*fin+x

##### Dual Attention Network

In [12]:
class Dual(nn.Module):
    def __init__(self):
        super(Dual,self).__init__()
        self.position = Position()
        self.channel = Channel() 
        self.conv=nn.Conv2d(4096,1,kernel_size=1)

    def forward(self,x):
        temp1=self.position(x)
        temp2=self.channel(x)
        temp_3=temp1+temp2
        return self.conv(temp_3)

In [ ]:
resnet50=models.resnet50(pretrained=True)
input_resize=torch.nn.functional.interpolate(images,size=(224,224))
print(input_resize.shape)
conv_layer = nn.Sequential(*list(resnet50.children())[:-2])
output_tensor = conv_layer(input_resize)
mx=nn.MaxPool2d(7)
output_tensor=mx(output_tensor)
flatten=nn.Flatten()
output_tensor=flatten(output_tensor)
linar=nn.Linear(2048,8*8*256)
output_tensor=linar(output_tensor)
output_tensor = output_tensor.view(100, 256, 8, 8)
print(output_tensor.shape)




In [13]:
dataiter = iter(trainloader)
images, labels = next(dataiter)


In [45]:
class MyResNet50(nn.Module):
    def __init__(self,in_channels):
        super(MyResNet50, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = nn.Sequential(*list(models.resnet50().children())[4])
        self.layer2 = nn.Sequential(*list(models.resnet50().children())[5])
        self.layer3 = nn.Sequential(*list(models.resnet50().children())[6])
        self.layer4 = nn.Sequential(*list(models.resnet50().children())[7])
        # self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        # self.fc = nn.Linear(2048, 1000)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        # x = self.avgpool(x)
        # x = x.view(x.size(0), -1)
        # x = self.fc(x)
        return x


resnet50_2 = MyResNet50(256) 

input_tensor = torch.randn(10, 256, 8, 8) 


output_tensor_2 = resnet50_2(input_tensor) 

print(output_tensor_2.shape) 


In [49]:
class MVSS(nn.Module):
    def __init__(self):
        super(MVSS,self).__init__()
        resnet50=models.resnet50(pretrained=True)
        self.conv_layer = nn.Sequential(*list(resnet50.children())[:-2])
        self.mx=nn.MaxPool2d(7)
        self.flatten=nn.Flatten()
        self.sobel_1=Sobel_layer(256,256)
        self.sobel_2=Sobel_layer(512,512)
        self.sobel_3=Sobel_layer(1024,1024)
        self.sobel_4=Sobel_layer(2048,2048)

        self.erb1=ERB(256,1)
        self.erb2=ERB(512,1)
        self.erb3=ERB(1024,1)
        self.erb4=ERB(2048,1)
        self.erb5=ERB(1,1)

        self.scale1 = nn.Upsample(scale_factor=2, mode="nearest")
        self.scale2 = nn.Upsample(scale_factor=4, mode="nearest")
        self.sig=nn.Sigmoid()

        self.bayar=BayarConv2d(3,3)

    def forward(self,x):

        b,c,h,w=x.size()
        input_resize=torch.nn.functional.interpolate(x,size=(224,224))
        output_1=self.conv_layer(input_resize)
        output_1=self.mx(output_1)
        output_1=self.flatten(output_1)
        linear_1=nn.Linear(2048,h//4*w//4*256)
        res_1=linear_1(output_1).view(10, 256, w//4, h//4)   
        # print(res_1.shape,  "Required 10, 256, 8, 8")
        sb_1=self.sobel_1(res_1)
        erb_1=self.erb1(sb_1)


        input_resize_2=torch.nn.functional.interpolate(res_1,size=(224,224))      
        resnet_2=MyResNet50(256)
        output_2=resnet_2(input_resize_2)
        output_2=self.flatten(output_2)
        linear_2=nn.Linear(2048,h//8*w//8*512)
        res_2=linear_2(output_1).view(10, 512, w//8, h//8)     
        # print(res_2.shape,  "Required 10, 512, 4, 4")
        sb_2=self.sobel_2(res_2)
        erb_2=self.erb2(sb_2)


        input_resize_3=torch.nn.functional.interpolate(res_2,size=(224,224))
        resnet_3=MyResNet50(512)
        output_3=resnet_3(input_resize_3)
        output_3=self.mx(output_3)
        output_3=self.flatten(output_3)
        linear_3=nn.Linear(2048,h//16*w//16*1024)
        res_3=linear_3(output_3).view(10, 1024, w//16, h//16)       
        # print(res_3.shape,  "Required 10, 1024, 2, 2")
        sb_3=self.sobel_3(res_3)
        erb_3=self.erb3(sb_3)


        input_resize_4=torch.nn.functional.interpolate(res_3,size=(224,224))
        resnet_4=MyResNet50(1024)
        output_4=resnet_4(input_resize_4)
        output_4=self.mx(output_4)
        output_4=self.flatten(output_4)
        linear_4=nn.Linear(2048,h//16*w//16*2048)
        res_4=linear_4(output_4).view(10, 2048, w//16, h//16)          
        # print(res_4.shape,  "Required 10, 2048, 2, 2")
        sb_4=self.sobel_4(res_4)
        erb_4=self.erb4(sb_4)


        erb_out1=erb_1+self.scale1(erb_2)
        erb_out_2=self.erb5(erb_out1)
        erb_out_3=self.scale2(erb_3)+erb_out_2
        erb_out_4=self.erb5(erb_out_3)
        erb_out_5=self.scale2(erb_4)+erb_out_4
        final_erb_out=self.erb5(erb_out_5)

        edge_out=self.sig(final_erb_out)


        

        return edge_out,res_4

        







In [50]:
mvss=MVSS()
mvss(images)

torch.Size([10, 1, 8, 8])


In [51]:
class MVSS(nn.Module):
    def __init__(self):
        super(MVSS,self).__init__()
        resnet50=models.resnet50(pretrained=True)
        self.conv_layer = nn.Sequential(*list(resnet50.children())[:-2])
        self.mx=nn.MaxPool2d(7)

        self.gmp=nn.MaxPool2d(512)

        self.flatten=nn.Flatten()
        self.sobel_1=Sobel_layer(256,256)
        self.sobel_2=Sobel_layer(512,512)
        self.sobel_3=Sobel_layer(1024,1024)
        self.sobel_4=Sobel_layer(2048,2048)

        self.erb1=ERB(256,1)
        self.erb2=ERB(512,1)
        self.erb3=ERB(1024,1)
        self.erb4=ERB(2048,1)
        self.erb5=ERB(1,1)

        self.scale1 = nn.Upsample(scale_factor=2, mode="nearest")
        self.scale2 = nn.Upsample(scale_factor=4, mode="nearest")
        self.sig=nn.Sigmoid()

        self.bayar=BayarConv2d(3,3)
        self.dual=Dual()



    def forward(self,x):

        b,c,h,w=x.size()
        input_resize=torch.nn.functional.interpolate(x,size=(224,224))
        output_1=self.conv_layer(input_resize)
        output_1=self.mx(output_1)
        output_1=self.flatten(output_1)
        linear_1=nn.Linear(2048,h//4*w//4*256)
        res_1=linear_1(output_1).view(10, 256, w//4, h//4)   
        # print(res_1.shape,  "Required 10, 256, 8, 8")
        sb_1=self.sobel_1(res_1)
        erb_1=self.erb1(sb_1)


        input_resize_2=torch.nn.functional.interpolate(res_1,size=(224,224))      
        resnet_2=MyResNet50(256)
        output_2=resnet_2(input_resize_2)
        output_2=self.flatten(output_2)
        linear_2=nn.Linear(2048,h//8*w//8*512)
        res_2=linear_2(output_1).view(10, 512, w//8, h//8)     
        # print(res_2.shape,  "Required 10, 512, 4, 4")
        sb_2=self.sobel_2(res_2)
        erb_2=self.erb2(sb_2)


        input_resize_3=torch.nn.functional.interpolate(res_2,size=(224,224))
        resnet_3=MyResNet50(512)
        output_3=resnet_3(input_resize_3)
        output_3=self.mx(output_3)
        output_3=self.flatten(output_3)
        linear_3=nn.Linear(2048,h//16*w//16*1024)
        res_3=linear_3(output_3).view(10, 1024, w//16, h//16)       
        # print(res_3.shape,  "Required 10, 1024, 2, 2")
        sb_3=self.sobel_3(res_3)
        erb_3=self.erb3(sb_3)


        input_resize_4=torch.nn.functional.interpolate(res_3,size=(224,224))
        resnet_4=MyResNet50(1024)
        output_4=resnet_4(input_resize_4)
        output_4=self.mx(output_4)
        output_4=self.flatten(output_4)
        linear_4=nn.Linear(2048,h//16*w//16*2048)
        res_4=linear_4(output_4).view(10, 2048, w//16, h//16)          
        # print(res_4.shape,  "Required 10, 2048, 2, 2")
        sb_4=self.sobel_4(res_4)
        erb_4=self.erb4(sb_4)


        erb_out1=erb_1+self.scale1(erb_2)
        erb_out_2=self.erb5(erb_out1)
        erb_out_3=self.scale2(erb_3)+erb_out_2
        erb_out_4=self.erb5(erb_out_3)
        erb_out_5=self.scale2(erb_4)+erb_out_4
        final_erb_out=self.erb5(erb_out_5)

        edge_out=self.sig(final_erb_out)

        bayar_out=self.bayar(x)

        input_resize_b1=torch.nn.functional.interpolate(bayar_out,size=(224,224))
        output_1=self.conv_layer(input_resize_b1)
        output_1=self.mx(output_1)
        output_1=self.flatten(output_1)
        linear_1=nn.Linear(2048,h//4*w//4*256)
        res_11=linear_1(output_1).view(10, 256, w//4, h//4)   
      


        input_resize_b2=torch.nn.functional.interpolate(res_11,size=(224,224))      
        resnet_2=MyResNet50(256)
        output_2=resnet_2(input_resize_b2)
        output_2=self.flatten(output_2)
        linear_2=nn.Linear(2048,h//8*w//8*512)
        res_22=linear_2(output_1).view(10, 512, w//8, h//8)     
      


        input_resize_b3=torch.nn.functional.interpolate(res_22,size=(224,224))
        resnet_3=MyResNet50(512)
        output_3=resnet_3(input_resize_b3)
        output_3=self.mx(output_3)
        output_3=self.flatten(output_3)
        linear_3=nn.Linear(2048,h//16*w//16*1024)
        res_33=linear_3(output_3).view(10, 1024, w//16, h//16)       
        


        input_resize_b4=torch.nn.functional.interpolate(res_33,size=(224,224))
        resnet_4=MyResNet50(1024)
        output_4=resnet_4(input_resize_b4)
        output_4=self.mx(output_4)
        output_4=self.flatten(output_4)
        linear_4=nn.Linear(2048,h//16*w//16*2048)
        res_44=linear_4(output_4).view(10, 2048, w//16, h//16)          
       
        dual_concat=torch.cat([res_4,res_44],dim=1)

        dual_out=self.dual(dual_concat)

        final_dual=self.sig(dual_out)

        global_max_val=self.gmp(final_dual)

        return edge_out,final_dual,global_max_val

        





